# Part3: MobileBERT for Text Prediction

In [ ]:
from gluonnlp.utils.parameter import count_parameters
from gluonnlp.utils.preprocessing import get_trimmed_lengths
from mxnet.gluon.data import batchify as bf
from mxnet.gluon.data import DataLoader
from mxnet.lr_scheduler import PolyScheduler